In [1]:
import semantic_kernel as sk
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
print(os.getenv("OPENAI_API_BASE"))


https://openaiswedenalk.openai.azure.com/


In [2]:
# for this example I want a valid logger
import logging

# Create a logger
logger = logging.getLogger("SK")

# Set the logging level to INFO
logger.setLevel(logging.DEBUG)

# Create a console handler
console_handler = logging.StreamHandler()

# Set the level of the console handler to INFO
console_handler.setLevel(logging.DEBUG)

# Create a formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Set the formatter for the console handler
console_handler.setFormatter(formatter)

# Add the console handler to the logger
logger.addHandler(console_handler)


In [4]:
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextCompletion,
)

kernel = sk.Kernel(log=logger)
kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(
        "GPT4t",
        endpoint = os.getenv("OPENAI_API_BASE"),
        api_key = os.getenv("OPENAI_API_KEY")
    ),
)

In [5]:
# Now we need to import the plugin
from plugins.AudioVideoPlugin.AudioVideo import AudioVideo

# Now you can import the plugin importing skill directly from the function you declared
# in the plugin directory. The import_skill does not need the path, it only need an
# instance of the skill and the name of the skill
extractaudio_plugin = kernel.import_skill(AudioVideo(), skill_name="AudioVideoPlugin")

plugins_directory = "./plugins"

# Import the OrchestratorPlugin from the plugins directory.
publishing_plugin = kernel.import_semantic_skill_from_directory(
    plugins_directory, "PublishingPlugin"
)


2023-12-05 18:06:36,058 - SK - DEBUG - Importing skill AudioVideoPlugin
2023-12-05 18:06:36,060 - SK - DEBUG - Methods imported: 2
2023-12-05 18:06:36,069 - SK - DEBUG - Extracting blocks from template: I will give you a transcript of a video. The transcript contains phrases prefixed by the timestamp where the phrase starts. I want you to identify between three and ten main sections of the video. You must never identify more than ten sections.
For each section you will create a brief title prefixed with the start timestamp of the section obtained analyzing all the text belonging to that section.

EXAMPLE ANSWER - Maximum of ten sections
00:00 - Title of section 1
00:33 - Title of section 2
01:23 - Title of section 3

[DATA]
{{$input}}


In [6]:
from pprint import pprint
# want to print all the keys of extractaudio_plugin that is a dictionary

pprint (extractaudio_plugin["ExtractAudio"])
pprint (extractaudio_plugin["TranscriptTimeline"])
pprint (publishing_plugin["VideoTimelineCreator"])

In [7]:
# you can verify if cuda is available.
import torch
torch.cuda.is_available()

True

In [8]:
# Now we can use the skill
# Run the Sqrt function with the context.
result = await kernel.run_async(
    extractaudio_plugin["ExtractAudio"],
    extractaudio_plugin["TranscriptTimeline"],
    publishing_plugin["VideoTimelineCreator"],
    input_str="C:\\temp\\230Github.mp4",
)


Extracting auio file from video C:\temp\230Github.mp4
Extracting transcript from audio file C:\temp\230Github.wav
Using device: cuda:0 to run whisper


100%|██████████| 63281/63281 [01:52<00:00, 564.45frames/s]
2023-12-05 18:08:47,643 - SK - DEBUG - Rendering string template: I will give you a transcript of a video. The transcript contains phrases prefixed by the timestamp where the phrase starts. I want you to identify between three and ten main sections of the video. You must never identify more than ten sections.
For each section you will create a brief title prefixed with the start timestamp of the section obtained analyzing all the text belonging to that section.

EXAMPLE ANSWER - Maximum of ten sections
00:00 - Title of section 1
00:33 - Title of section 2
01:23 - Title of section 3

[DATA]
{{$input}}
2023-12-05 18:08:47,643 - SK - DEBUG - Extracting blocks from template: I will give you a transcript of a video. The transcript contains phrases prefixed by the timestamp where the phrase starts. I want you to identify between three and ten main sections of the video. You must never identify more than ten sections.
For each section

extracted 114 audio segments


In [9]:
print(result)

00:00 - Introduction to Secure GitHub Access with YubiKey
00:38 - Cloning and Interacting with GitHub Repositories via SSH
02:32 - Creating an SSH Key Inside YubiKey on a Remote Desktop
04:21 - Adding the Public Key to GitHub Account
05:39 - Cloning a Repository Using SSH Protocol
07:14 - Advantages of SSH Protocol for Security and Authentication
09:20 - Benefits of Using SSH with YubiKey for Remote Desktop Users
10:16 - Conclusion and Benefits of SSH Key with YubiKey for GitHub Security
